# Migration from Los Angeles County

### Import python tools

In [1]:
import pandas as pd
import jenkspy 
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import json
import os
import seaborn as sns
import altair_latimes as lat
import altair as alt
from pandas.io.json import json_normalize 
alt.renderers.enable('notebook')
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Read data downloaded from IRS

In [2]:
# source_files = sorted(Path('/Users/mhustiles/data/github/notebooks/migration/input/irs/').glob('*.csv'))

# dataframes = []
# for f in source_files:
#     df = pd.read_csv(f, encoding = "ISO-8859-1")
#     df['source'] = f.name
#     df['source'] = df['source'].str.replace('countyoutflow', '').str.replace('.csv', '', regex=False)
#     dataframes.append(df)

# src = pd.concat(dataframes, sort=False)

In [3]:
df_1011 = pd.read_csv('input/irs/countyoutflow1011.csv', sep=',', encoding='latin-1')
df_1112 = pd.read_csv('input/irs/countyoutflow1112.csv', sep=',', encoding='latin-1')
df_1213 = pd.read_csv('input/irs/countyoutflow1213.csv', sep=',', encoding='latin-1')
df_1314 = pd.read_csv('input/irs/countyoutflow1314.csv', sep=',', encoding='latin-1')
df_1415 = pd.read_csv('input/irs/countyoutflow1415.csv', sep=',', encoding='latin-1')
df_1516 = pd.read_csv('input/irs/countyoutflow1516.csv', sep=',', encoding='latin-1')

In [4]:
src = pd.concat([df_1011,df_1112,df_1213,df_1314,df_1415,df_1516], sort=False)

### Process data types and fields

In [5]:
all_years = src.astype({'y1_statefips': 'str',\
                        'y1_countyfips': 'str',\
                        'y2_statefips': 'str',\
                        'y2_countyfips': 'str'}, inplace=True)

In [6]:
all_years['y1_statefips'] = all_years['y1_statefips'].str.zfill(2)
all_years['y1_countyfips'] = all_years['y1_countyfips'].str.zfill(3)
all_years['y2_statefips'] = all_years['y2_statefips'].str.zfill(2)
all_years['y2_countyfips'] = all_years['y2_countyfips'].str.zfill(3)

In [7]:
all_years['y1_fips'] = all_years['y1_statefips'] + all_years['y1_countyfips']
all_years['y2_fips'] = all_years['y2_statefips'] + all_years['y2_countyfips']
all_years['geoid'] = all_years['y2_fips']

In [8]:
all_years.head(10)

,y1_statefips,y1_countyfips,y2_statefips,y2_countyfips,y2_state,y2_countyname,n1,n2,agi,y1_fips,y2_fips,geoid
0,00,000,96,000,US,Total Mig - US & For,6991456,12967005,304464050,00000,96000,96000
1,00,000,97,000,US,Total Mig - US,6890106,12775807,299138260,00000,97000,97000
2,00,000,97,001,US,Total Mig - US Same St,3916380,7260905,157646960,00000,97001,97001
3,00,000,97,003,US,Total Mig - US Diff St,2973726,5514902,141491300,00000,97003,97003
4,00,000,98,000,US,Total Mig - Foreign,101350,191198,5325790,00000,98000,98000
5,01,000,96,000,AL,Total Mig - US & For,99179,202679,3665124,01000,96000,96000
6,01,000,97,000,AL,Total Mig - US,98140,200358,3620373,01000,97000,97000
7,01,000,97,001,AL,Total Mig - US Same St,54577,112357,1832756,01000,97001,97001
8,01,000,97,003,AL,Total Mig - US Diff St,43563,88001,1787617,01000,97003,97003
9,01,000,98,000,AL,Total Mig - Foreign,1039,2321,44751,01000,98000,98000


### Map year 1 FIPS codes

In [9]:
y2 = all_years[['y2_fips','y2_countyname','y2_state']]

In [10]:
fipsgroup = y2.groupby(['y2_fips','y2_countyname','y2_state'])

### Filter national aggregates and people who stayed in LAC

In [11]:
aggregates = ['57', '58', '59', '96', '97', '98']

In [12]:
la_in = all_years[\
#                       (all_years['y2_state'] != 'NY') &\
#                       (all_years['y2_state'] != 'NJ') &\
                      (all_years['y2_fips'] == '06037') &\
                      (all_years['y1_fips'] != '06037') &\
                      (~all_years.y2_statefips.isin(aggregates))]

In [13]:
la_in.head()

,y1_statefips,y1_countyfips,y2_statefips,y2_countyfips,y2_state,y2_countyname,n1,n2,agi,y1_fips,y2_fips,geoid
106,01,003,06,037,CA,Los Angeles County,11,19,518,01003,06037,06037
897,01,073,06,037,CA,Los Angeles County,76,114,2576,01073,06037,06037
1281,01,089,06,037,CA,Los Angeles County,34,52,1444,01089,06037,06037
1490,01,097,06,037,CA,Los Angeles County,35,59,1130,01097,06037,06037
1663,01,101,06,037,CA,Los Angeles County,19,36,508,01101,06037,06037


In [14]:
aggregated = all_years[all_years.y2_statefips.isin(aggregates)]

In [15]:
# total = pd.DataFrame(aggregated[(aggregated['y2_countyname'].str.contains('Total Migration-Different State')) &\
#                    (aggregated['y2_statefips'] != '96') &\
#                   (aggregated['y1_countyfips'] != '000')])

In [16]:
# total['merge_fips_norm'] = total['y1_statefips'] + total['y1_countyfips']

In [17]:
# normalization_totals = total.groupby(['merge_fips_norm']).agg({'n1':'sum'}).reset_index()

### Group by locations where tax filers moved

In [18]:
migration = la_in.groupby(['y1_fips', 'geoid']).agg({'n1':'sum'}).reset_index()

In [19]:
migration.rename(columns={'n1':'tax_migrants_to_lac'}, 
                 inplace=True)

In [20]:
migration.head(5)

,y1_fips,geoid,tax_migrants_to_lac
0,01003,06037,58
1,01069,06037,12
2,01073,06037,423
3,01081,06037,57
4,01089,06037,252


### How many tax filers who left LAC in total?

In [21]:
migration.tax_migrants_to_lac.sum()

647449

### How many tax filers who left LAC moved to each state?

In [ ]:
states = migration.groupby(['y1_fips']).agg('sum').reset_index()

In [27]:
states.sort_values(by='tax_migrants_to_lac',ascending=False).head(10)

,y1_fips,tax_migrants_to_lac
58,06059,96859
64,06071,68640
61,06065,39617
65,06073,30482
82,06111,26255
375,32003,17475
45,06029,14333
432,36061,14128
17,04013,13046
198,17031,11459


In [28]:
migration.sort_values(by='tax_migrants_to_lac',ascending=False).head(10)

,y1_fips,geoid,tax_migrants_to_lac
58,06059,06037,96859
64,06071,06037,68640
61,06065,06037,39617
65,06073,06037,30482
82,06111,06037,26255
375,32003,06037,17475
45,06029,06037,14333
432,36061,06037,14128
17,04013,06037,13046
198,17031,06037,11459


In [ ]:
normal_migration = pd.merge(migration, normalization_totals, left_on='geoid', right_on='merge_fips_norm', how='inner')

In [ ]:
normal_migration.head()

In [ ]:
normal_migration.rename(columns={'n1':'all_tax_migrants_to_county'}, 
                 inplace=True)

In [ ]:
normal_migration.head()

In [ ]:
normal_migration['rate_per_1000'] = \
    (normal_migration['tax_migrants_from_lac'] / normal_migration['all_tax_migrants_to_county'])*1000

In [ ]:
normal_migration['pct_all_migrants_from_lac'] = \
    (normal_migration['tax_migrants_from_lac'] / 740469)*100

In [ ]:
normal_migration.sort_values(by='rate_per_1000', ascending=False).head(10)

In [ ]:
normal_migration.sort_values(by='pct_all_migrants_from_lac', ascending=False).head(10)

### Prepare natural breaks for mapping

In [ ]:
breaks = jenkspy.jenks_breaks(list(normal_migration.rate_per_1000), nb_class=7)

In [ ]:
breaks

In [ ]:
def get_group(value):
    for i, b in enumerate(breaks):
        if value <= breaks[i+1]:
            return i

### Attach break groups to dataframe

In [ ]:
normal_migration['mover_group'] = normal_migration.rate_per_1000.apply(get_group)

In [ ]:
normal_migration.sort_values(by='rate_per_1000',ascending=False).head(20)

---

### Import U.S. counties geography

In [ ]:
counties = gpd.read_file('/Users/mhustiles/data/data/GIS/usa/counties_lakes.shp')

In [ ]:
counties.columns = counties.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

### Remove Alaska and Hawaii

In [ ]:
lower48 = counties[(counties['statefp'] != '15') & (counties['statefp'] != '02')]

In [ ]:
lower48.plot()

In [ ]:
lower48.head()

---

### Merge geography with migration data

In [ ]:
migration_map = pd.merge(lower48, normal_migration, on='geoid')

In [ ]:
migration_map.loc[0]

In [ ]:
slim = migration_map[['geoid','name','mover_group','rate_per_1000','geometry']]

In [ ]:
slim.plot()

In [ ]:
geojson = json.loads(slim.to_json())
features = alt.Data(values=geojson['features'])

In [ ]:
background = alt.Chart(features).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    width=800,
    height=500
).project('albersUsa')

In [ ]:
base = alt.Chart(features).mark_geoshape(
    fill='lightgray',
    strokeWidth=0.1,
    stroke='white'
).properties(
    width=800,
    height=500
).project('albersUsa')

In [ ]:
geoshape = alt.Chart(features).mark_geoshape(
    fill='lightgray',
    stroke='white'
)

In [ ]:
movers = geoshape.encode(
    color=alt.Color(
        "properties.mover_group:N",
        scale=alt.Scale(
            domain=[0,1, 2, 3, 4, 5, 6],
            range=lat.palette['schemes']['fire-7']
        ),
        legend=None
    ),
)

In [ ]:
(base + movers).properties(title="LA County out migration: 2010-2015").configure_view(
    strokeWidth=0 
)